In [2]:
#import os
#import sys
#from collections import defaultdict
#from itertools import groupby
#from random import shuffle
#from collections import namedtuple
#import codecs
import multiprocessing
import csv

import nltk
#import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.cluster import DBSCAN

import utils
from utils import tokenizer, stopwords, stemmer

In [26]:
from scipy.spatial.distance import cosine

In [6]:
# largest distance between points for clustering:
eps = 0.1
include_title = False
include_description = True

# TODO:
title_weight = 0.5
description_weight = 1 - title_weight


def tokenize_and_stem(text):
    """
    Tokenize and stem English text
    """
    global stemmer, tokenizer
    return [stemmer.stem(token) for token in tokenizer.tokenize(text)]

In [109]:
def ilines(file1_name, file2_name, titles=False):
    """
    File line iterator
    Yields "raw" documents
    """
    global jo_ids, last_target_row, first_source_row

    # Line no -> code
    jo_ids = []
    
    # TARGET:
    with open(file2_name, mode='r') as fileobj:
        rdr = csv.reader(fileobj)

        for i, row in enumerate(rdr):
            if i == 0: ## skip header
                continue
            # industry, occ_code, occ_name, occ_descriptions
            # sample_job_titles, task1, task2, task3, task4, task5
            jo_ids.append(row[1])
            # include industry, occ_name, occ_descriptions
            # task1, task2, task3, task4, task5
            #res = row[0] + ' ' + row[2] + ' ' + row[3] + ' ' + ' '.join(row[5:])
            res = row[3]
            yield res
        last_target_row = i-1 # row #0 is the header row
        first_source_row = i # row #0 is the header row

    # SOURCE:
    with open(file1_name, mode='r') as fileobj:
        rdr = csv.reader(fileobj)

        for i, row in enumerate(rdr):
            jo_ids.append(i)
            res = row[1]
            yield res

In [132]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(
    #input="filename",
    #max_df=1.1,
    #max_features=200000,
    #min_df=0.01,
    lowercase=True,
    analyzer='word',
    stop_words=stopwords,
    use_idf=True,
    tokenizer=tokenize_and_stem,
    ngram_range=(1,2))

In [133]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(ilines("data/File 1.csv", "data/File 2.csv"))
print("*** Vector space shape:", tfidf_matrix.shape)

*** Vector space shape: (4351, 62646)
CPU times: user 2.95 s, sys: 12 ms, total: 2.96 s
Wall time: 3.05 s


/data/Match-Job-Occupations-Textual-Analysis-IK-Research/venv/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [112]:
from sklearn.metrics.pairwise import cosine_similarity

In [134]:
similarity = cosine_similarity(tfidf_matrix[first_source_row:,:], tfidf_matrix[:first_source_row,:])

/data/Match-Job-Occupations-Textual-Analysis-IK-Research/venv/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [138]:
def take_top10(row):
    return sorted([r for r in zip(range(row.size), row.tolist()) if r[1] > 0.1],
           key=lambda r: -r[1])[:10]

In [139]:
top10 = []
for row in similarity:
    top10.append(take_top10(row))

In [136]:
top10[:10]

[[], [], [], [], [], [], [], [], [], []]

In [140]:
for i, l in enumerate(top10, 1):
    if l != []:
        print("Line {}: ".format(i),  end="")
        print([k+2 for (k, _) in l])
    #else:
    #    print("NO MATCH")

Line 3: [653, 1784]
Line 14: [58, 391]
Line 15: [1575]
Line 16: [731, 1589, 137, 598, 90, 1602, 2076, 846, 1474, 1632]
Line 18: [230, 2012, 205]
Line 19: [822, 2197, 2223]
Line 23: [755, 1368, 1710, 1311, 1697]
Line 28: [1315, 1658, 2282, 1151, 1400, 1608, 647, 1778, 1313, 1628]
Line 32: [1276, 2029, 208, 2080]
Line 37: [302]
Line 40: [302]
Line 43: [750, 1363, 1705, 245, 831, 2136, 86, 1148, 1752, 871]
Line 44: [307, 289, 295, 299, 304, 297, 308, 323, 520, 1167]
Line 50: [101, 429, 1091]
Line 52: [854, 1317, 1719]
Line 62: [266, 1516, 2279, 2341]
Line 63: [315, 319]
Line 64: [302]
Line 75: [180, 1126, 1559, 2039]
Line 76: [1224]
Line 77: [875, 1322, 1646, 690, 1376, 1741, 689, 1375, 1740]
Line 79: [1259, 107, 1258, 1262, 731, 1589, 123, 1947, 1261, 124]
Line 80: [1259, 1258, 1262, 107, 731, 1589, 124, 1948, 717, 1337]
Line 83: [648, 1779, 1311, 1697]
Line 87: [646, 1777]
Line 88: [646, 1777]
Line 98: [870, 1962]
Line 102: [1165, 1609, 538, 1150, 1612, 644, 1775]
Line 108: [1165, 1609,

In [94]:
?tfidf_vectorizer.fit

In [96]:
tfidf_vectorizer.transform("""
"Design, develop, or evaluate energy-related projects or programs to reduce energy costs or improve energy efficiency during the designing, building, or remodeling stages of construction. May specialize in electrical systems; heating, ventilation, and air-conditioning (HVAC) systems; green buildings; lighting; air quality; or energy procurement.","Sample of reported job titles: Distributed Generation Project Manager, Energy Efficiency Engineer, Energy Engineer, Energy Manager, Environmental Solutions Engineer, Industrial Energy Engineer, Measurement And Verification Engineer, Test and Balance Engineer",Identify energy savings opportunities and make recommendations to achieve more energy efficient operation.,"Manage the development, design, or construction of energy conservation projects to ensure acceptability of budgets and time lines, conformance to federal and state laws, or adherence to approved specifications.","Conduct energy audits to evaluate energy use, costs, or conservation measures.",Monitor and analyze energy consumption.,"Perform energy modeling, measurement, verification, commissioning, or retro-commissioning."
""")

/data/Match-Job-Occupations-Textual-Analysis-IK-Research/venv/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


<1145x10500 sparse matrix of type '<class 'numpy.float64'>'
	with 733 stored elements in Compressed Sparse Row format>